<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обработка данных</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span><ul class="toc-item"><li><span><a href="#После-ресемплинга-с-уменьшением-класса-0" data-toc-modified-id="После-ресемплинга-с-уменьшением-класса-0-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>После ресемплинга с уменьшением класса 0</a></span></li><li><span><a href="#Автоматическая-настройка-весов-'balanced'" data-toc-modified-id="Автоматическая-настройка-весов-'balanced'-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Автоматическая настройка весов 'balanced'</a></span></li></ul></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>DecisionTree</a></span><ul class="toc-item"><li><span><a href="#После-ресемплинга-с-уменьшением-класса-0" data-toc-modified-id="После-ресемплинга-с-уменьшением-класса-0-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>После ресемплинга с уменьшением класса 0</a></span></li><li><span><a href="#Автоматическая-настройка-весов-'balanced'" data-toc-modified-id="Автоматическая-настройка-весов-'balanced'-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Автоматическая настройка весов 'balanced'</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span><ul class="toc-item"><li><span><a href="#Проверка-на-адекватность" data-toc-modified-id="Проверка-на-адекватность-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Проверка на адекватность</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>LogisticRegression</a></span><ul class="toc-item"><li><span><a href="#После-ресемплинга-с-уменьшением-класса-0" data-toc-modified-id="После-ресемплинга-с-уменьшением-класса-0-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>После ресемплинга с уменьшением класса 0</a></span></li><li><span><a href="#Автоматическая-настройка-весов-'balanced'" data-toc-modified-id="Автоматическая-настройка-весов-'balanced'-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Автоматическая настройка весов 'balanced'</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

**Цель:** построить модель со значением метрики качества *AUC-ROC* не меньше 0.85.

**Описание данных**

Данные состоят из файлов, полученных из разных источников. Во всех файлах столбец customerID содержит код клиента. Информация о договорах актуальна на 1 февраля 2020.

## Подготовка

In [1]:
import pandas as pd 
import numpy as np 
import warnings
import re
import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

warnings.filterwarnings("ignore")

In [2]:
try:
    data = pd.read_csv("D:/Users/Egor/Desktop/Proga/Jupyter/toxic/toxic_comments.csv", index_col=0)
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
data.sample(10)

,text,toxic
27235,"""[outdent]David, I have tried in the nicest p...",0
85350,"This is Spitfire \n\nI'm a giant douche, that'...",1
17437,Please don't tell me what to do. The discussi...,0
141674,Hello.\nSo like a month back I wasn't behaving...,0
15423,"""\n\n Pro-German \n\nI think the article shoul...",0
158932,"""\nInactive bureaucrat?\nShould bureaucrats th...",0
117484,I had posted this as an edit description but i...,0
134468,"Went up several hours ago sorry, it's at Gaza_...",0
148579,devout Christian and politically conservative....,0
29928,Don't be dense. -user,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [5]:
data.duplicated().sum()

0

In [6]:
data.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

**Вывод:**
- Данные загружены;
- Пропущенных значений и дубликатов нет;
- В целевом признаке около 90% объектов отрицательного класса.

## Обработка данных

In [7]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,             
                "N": wordnet.NOUN,              
                "V": wordnet.VERB,              
                "R": wordnet.ADV                
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def get_lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [8]:
#data.to_csv('lemm_toxic.csv', index=False)
try:
    data = pd.read_csv("D:/Users/Egor/Desktop/Proga/Jupyter/toxic/lemm_toxic.csv")
except:  
    %%time
    data['text'] = data['text'].apply(clear_text)
    data['text'] = data['text'].apply(get_lemm_text) 
data.head(20)

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0
5,congratulation from me a well use the tool wel...,0
6,cocksucker before you piss around on my work,1
7,your vandalism to the matt shirvington article...,0
8,sorry if the word nonsense be offensive to you...,0
9,alignment on this subject and which be contrar...,0


In [9]:
features = data.drop(['toxic'], axis=1) 
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.2, random_state=12345, stratify = target)

In [10]:
print(f'Размерность признаков - {features_train.shape} и целевого признака - {target_train.shape} обучающей выборки')
print(f'Размерность признаков - {features_test.shape} и целевого признака - {target_test.shape} тестовой выборки')

Размерность признаков - (127433, 1) и целевого признака - (127433,) обучающей выборки
Размерность признаков - (31859, 1) и целевого признака - (31859,) тестовой выборки


In [13]:
nltk.download("stopwords")
stopwords = set(nltk_stopwords.words("english"))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_without_unicode = count_tf_idf.fit_transform(features_train['text'].apply(lambda x: np.str_(x)))
features_test_without_unicode = count_tf_idf.transform(features_test['text'].apply(lambda x: np.str_(x)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Egor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
print(f'Размерность признаков - {features_train_without_unicode.shape} и целевого признака - {target_train.shape} обучающей выборки')
print(f'Размерность признаков - {features_test_without_unicode.shape} и целевого признака - {target_test.shape} тестовой выборки')

Размерность признаков - (127433, 132707) и целевого признака - (127433,) обучающей выборки
Размерность признаков - (31859, 132707) и целевого признака - (31859,) тестовой выборки


In [15]:
features_train = features_train_without_unicode
features_test = features_test_without_unicode

In [16]:
print('До уменьшения класса 0:')
target_train.value_counts(normalize=True)

До уменьшения класса 0:


0    0.898386
1    0.101614
Name: toxic, dtype: float64

In [17]:
train = data.iloc[target_train.index]

target_train_0 = train[train['toxic'] == 0]['toxic']
target_train_1 = train[train['toxic'] == 1]['toxic']

target_train_0_downsample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_downsample = pd.concat([target_train_0_downsample, target_train_1])

features_train_downsample = data.iloc[target_train_downsample.index]

features_train_downsample, target_train_downsample = shuffle(features_train_downsample, target_train_downsample, random_state=12345)
features_train_downsample = count_tf_idf.transform(features_train_downsample['text'].apply(lambda x: np.str_(x)))

In [18]:
print('После ресемплинга с уменьшением класса 0:')
target_train_downsample.value_counts(normalize=True)

После ресемплинга с уменьшением класса 0:


1    0.5
0    0.5
Name: toxic, dtype: float64

**Вывод:**
- Текст прошел через функцию для очистки и лемматизации;
- Данные разделены на тренировочную, валидационную и тестовую выборки в отношении 3:1:1;
- Проведена векторизацию данных;
- В обучающей выборке *downsample* находятся сбалансированные классы.

## Обучение

### LogisticRegression

#### После ресемплинга с уменьшением класса 0

In [19]:
%%time
logmodel = LogisticRegression(random_state=12345) 

param = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'], 'C':[2,4,5,6,8,10]}
gscv_log = GridSearchCV(logmodel, param, scoring='f1', cv=3, n_jobs=-1)
gscv_log.fit(features_train_downsample, target_train_downsample)

print(gscv_log.best_params_)

{'C': 4, 'solver': 'saga'}
Wall time: 46.9 s


#### Автоматическая настройка весов 'balanced'

In [20]:
%%time
logmodel = LogisticRegression(random_state=12345) 

param = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'], 'C':[2,4,5,6,8,10], 'class_weight':['balanced']}
gscv_log_auto = GridSearchCV(logmodel, param, scoring='f1', cv=3, n_jobs=-1)
gscv_log_auto.fit(features_train, target_train)

print(gscv_log_auto.best_params_)

{'C': 10, 'class_weight': 'balanced', 'solver': 'saga'}
Wall time: 3min 32s


### CatBoost

In [21]:
%%time
catmodel = CatBoostClassifier(verbose=False) 

param = {'iterations':[200]}
gscv_cat = GridSearchCV(catmodel, param, scoring='f1', cv=3)
gscv_cat.fit(features_train_downsample, target_train_downsample)

print(gscv_cat.best_params_)

{'iterations': 200}
Wall time: 4min 54s


### DecisionTree

#### После ресемплинга с уменьшением класса 0

In [22]:
%%time
treemodel = DecisionTreeClassifier(random_state=12345)
param = {'max_depth': [x for x in range(1, 25, 2)]}
gscv_tree = GridSearchCV(treemodel, param, scoring='f1', cv=3)
gscv_tree.fit(features_train_downsample, target_train_downsample)

print(gscv_tree.best_params_)

{'max_depth': 23}
Wall time: 1min 13s


#### Автоматическая настройка весов 'balanced'

In [23]:
%%time
treemodel = DecisionTreeClassifier(random_state=12345)
param = {'max_depth': [x for x in range(1, 25, 2)], 'class_weight':['balanced']}
gscv_tree_auto = GridSearchCV(treemodel, param, scoring='f1', cv=3)
gscv_tree_auto.fit(features_train, target_train)

print(gscv_tree_auto.best_params_)

{'class_weight': 'balanced', 'max_depth': 23}
Wall time: 12min 12s


In [24]:
df = [[gscv_log.best_score_],
      [gscv_log_auto.best_score_],
      [gscv_cat.best_score_], 
      [gscv_tree.best_score_],
      [gscv_tree_auto.best_score_]]
model = ["LogisticRegression", "LogisticRegression ('balanced')", "CatBoost", "DecisionTree", "DecisionTree ('balanced')"]
pd.DataFrame(data=df, index=model, columns=["f1-score"])

,f1-score
LogisticRegression,0.888842
LogisticRegression ('balanced'),0.758892
CatBoost,0.872715
DecisionTree,0.710971
DecisionTree ('balanced'),0.615863


1. Среди моделей с хорошей метрикой f1-score можно выделить - CatBoost, LogisticRegression.
2. Хуже всего себя показала модель DecisionTree, ее оценка f1 составляет ~ 0.61 и 0.71. Ее точно не стоит рассматривать как финальную.
3. LogisticRegression показала себя средне с результатом f1-score = 0.75.

Выбираем **LogisticRegression**, так как среди всех моделей она показала лучшую оценку f1-score большую 0.75.

## Тестирование

### Проверка на адекватность

In [25]:
dummy = DummyClassifier(strategy="constant", constant=1.0)
dummy.fit(features_train, target_train)
print('F1 для константной модели:', f1_score(target_test, dummy.predict(features_test)))

F1 для константной модели: 0.18446546614998857


### LogisticRegression

#### После ресемплинга с уменьшением класса 0

In [26]:
logmodel = LogisticRegression(random_state=12345, n_jobs=-1, C=4, solver='saga')

logmodel.fit(features_train_downsample, target_train_downsample)
log_predict = logmodel.predict(features_test)
log_test_f1 = f1_score(target_test, log_predict)

print('F1 на тестовой выборке:', log_test_f1)

F1 на тестовой выборке: 0.7088858594621391


#### Автоматическая настройка весов 'balanced'

In [27]:
logmodel = LogisticRegression(random_state=12345, n_jobs=-1, C=10, solver='saga', class_weight='balanced')

logmodel.fit(features_train, target_train)
log_auto_predict = logmodel.predict(features_test)
log_auto_test_f1 = f1_score(target_test, log_auto_predict)

print('F1 на тестовой выборке:', log_auto_test_f1)

F1 на тестовой выборке: 0.7729880829777844


## Выводы

Во время проекты были загружены данные, удален лишний столбец. Текст прошел через функцию для очистки и лемматизации. Данные разделены на тренировочную, валидационную и тестовую выборки. Проведена векторизацию данных.

1. Среди моделей с хорошей метрикой f1-score можно выделить - CatBoost, LogisticRegression.
2. Хуже всего себя показала модель DecisionTree, ее оценка f1 составляет ~ 0.61 и 0.71. Ее точно не стоит рассматривать как финальную.
3. LogisticRegression показала себя средне с результатом f1-score = 0.75.

Лучшей моделью стала **LogisticRegression** с автоматической настройкой весов *class_weight='balanced'* с f1-score равной 0.773 и параметрами *n_jobs=-1*, *solver='saga'* и *C=10*. Проверку на адекватность модель также прошла успешно (по предсказаниям по константе = 1.0, f1-score = 0.184).